# Introduction
This project will depend on traditional computer vision(CV) techniques for classification tasks. Traditional CV goes through the following steps:-<br>
* Key points extraction using SIFT, SURF, ORB, BRIEF, FAST, or HOG
* Calculating the descriptors for those key points
* Clustering those descriptors into k-clusters
* Create a histogram for each image with k-bins and then quantize those key points according to the clusters that they are closest to, we can use different similarity measures, like, L2-distance, L1-distance, or cosine and so on.
* Then use those histogram vectors for the machine learning step, we can use SVM, logistic regression or trees for the classification step

In [ ]:
#To ignore Sklearn warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
import cv2

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report, log_loss, confusion_matrix
from sklearn.preprocessing import PolynomialFeatures


In [ ]:
plt.rcParams["figure.figsize"] = (10, 10)

# Unzip The Training and Test Set

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/" + "train.zip","r") as z:
    z.extractall(".")
with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/" + "test.zip","r") as z:
    z.extractall(".")


# Loading and Transforming Images
I chose the SIFT extractor to get a descriptor vector of size 128, and I chose SIFT because it give better descriptors than ORB, SURF and BRIEF. But it isn't as good as the features that are generated by HOG. Also, I will utilize the bag of visual words concept (BoVW) to describe the features in which instead of depending on the 128 vector for classification, I will rely on the frequency of occurrence of each cluster in an image. Hence, I will generate k-clusters based on the k-means algorithm then will generate the vocabulary list that is going to be used by BOWImgDescriptorExtractor. But my implementation for this project necessitated that I can't rely on BOWImgDescriptorExtractor for descriptor extraction because I can't store the whole of the dataset on RAM, hence, I can't generate the ideal clusters. Hence, I implemented my own L2- norm distance function that calculate the distance from the descriptors to the clusters in which this will eventually participate in constructing my histogram for the number of occurrence of each cluster in an image. Hence, my feature vector that is used for classification is the histogram of the BoVW in which its dimension is equal to the number of clusters.

In [ ]:
class LoadingImages(object):


    def __init__(self, trainingSize, numClusters=10, nfeatures=10):
        self.counts = {
            "cat": 0,
            "dog": 0,
            "test": 0
        }
        #Use SIFT descriptors
        self.sift = cv2.SIFT_create(nfeatures, 3, 0.0, 10, 1.6, None)
        #Use FLANN as your matcher between the BOW for each cluster in the 128 parameters space.
        FLANN_INDEX_KDTREE = 1 
        flann_params = {"algorithm": FLANN_INDEX_KDTREE, "trees": 5}
        matcher = cv2.FlannBasedMatcher(flann_params, {})
        #Compute image descriptor using bag of word method
        self.bowExtractor = cv2.BOWImgDescriptorExtractor(self.sift, matcher)
        self.bowKmeansClusters = cv2.BOWKMeansTrainer(numClusters)
        self.nfeatures  = nfeatures
        self.numClusters = numClusters
        self.trainingSize = trainingSize
        self.indeces = list(np.random.choice(self.trainingSize, self.trainingSize, replace=False))

    def resetCounters(self):
        self.counts = {
            "cat": 0,
            "dog": 0,
            "test": 0
        }

    def loadData(self, numImages, windowSize, countType, p='', trainOrTest="train", showMe=False, start=0):
        images = []

        for c in range(start, numImages):

            try:
                if countType == "test":
                    ind = self.counts["test"] + 1
                else:
                    ind = self.indeces[self.counts[countType]]
                img = cv2.imread(f"/kaggle/working/{trainOrTest}/" + p + str(ind) + ".jpg", cv2.IMREAD_GRAYSCALE)

                #histogram equalization to remove skewness in the intensity
                img_center = np.zeros((windowSize, windowSize), dtype=np.ubyte)
                #print(img.shape)

                # Using strides
                # print(img.shape)
                # print(img_center.shape)
                numStrides = [int(np.ceil(img.shape[0]/windowSize)), int(np.ceil(img.shape[1]/windowSize ))]
                numStrides[0] = 2 if numStrides[0] <= 1 else numStrides[0]
                numStrides[1] = 2 if numStrides[1] <= 1 else numStrides[1]
                #print(numStrides)
                img = img[::numStrides[0], ::numStrides[1]]
                img_center[:img.shape[0], :img.shape[1]] = img[:, :]

                #Getting the middle part of the image
                # center = (int(img.shape[0]/2), int(img.shape[1]/2))
                # #Get a centered Verision of the image
                # ptLeftX = center[0] - int(windowSize/2) if center[0] - int(windowSize/2) > 0 else 0
                # ptLeftY = center[1] - int(windowSize/2) if center[1] - int(windowSize/2) > 0 else 0

                # ptRightX = center[0] + int(windowSize/2) if center[0] + int(windowSize/2) < img.shape[0] else img.shape[0]
                # ptRightY = center[1] + int(windowSize/2) if center[1] + int(windowSize/2) < img.shape[1] else img.shape[1]

                # img_center = img[ptLeftX:ptRightX, ptLeftY:ptRightY]
                
                
                img_center = cv2.equalizeHist(img_center)
                img_center = np.asarray(img_center, np.float32)
                means, stddev = cv2.meanStdDev(img_center)
                img_center -= means
                stddev = stddev if stddev != 0 else 1
                img_center /= stddev

                if showMe:
                    cv2.imshow(p + str(ind), img_center)
                    if cv2.waitKey(0) & 0xFF == ord('q'):
                        cv2.destroyAllWindows()
                        break
                    cv2.destroyAllWindows()
                #print(np.mean(img_center))
                #print(np.std(img_center))
                #print(img_center.shape)
                images.append(img_center)
                self.counts[countType] += 1


            except Exception as e:
                #Restart your image loading
                print(e)
                print(self.counts[countType])
                self.counts[countType] = start
                #images = []
                break
        return images

    def getDescriptors(self, images, labels, visualize=False, kps=[]):
        descriptors = []
        counter = 0
        for img in images:
            img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
            kp = self.sift.detect(img)
            kps.append(kp)
            des = np.zeros((self.nfeatures, 128))
            if visualize:
                channels = []
                colored = None
                channels.append(img)
                channels.append(img)
                channels.append(img)
                colored = cv2.merge(channels)
                colored = cv2.drawKeypoints(img, kp, colored, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                cv2.imshow(str(counter), colored)
                if cv2.waitKey(0) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    break
                cv2.destroyAllWindows()
            try:
                #tmp = np.ravel(self.sift.compute(img, kp)[-1][0:self.nfeatures]).reshape(1, -1)
                tmp = self.sift.compute(img, kp)[-1]
                des[:len(tmp)] = tmp  
            except:
                pass

            #Make feature vector 1x(128 * #kp)    
            descriptors.append(des)
            counter += 1
        #Return #images x #features x 128, #imagesx x 1

        return descriptors, labels

    def getBOW(self, descriptors, labels, trainFlag=False):
        #add our data to cluster the visual vocabularly on
        #labels_extends = []
        if trainFlag:
            #nfeaturex128
            for arr in descriptors:
                #print(arr.shape)
                self.bowKmeansClusters.add(arr)#This will keep previous data unless we clear the data structure
            vocuabulary = self.bowKmeansClusters.cluster()
            #print(vocuabulary)
            #After finding vocabulary for each cluster, we need to set the BOW for those clusters
            self.bowExtractor.setVocabulary(vocuabulary)
        
        #print(self.bowExtractor.getVocabulary())
        vocuabulary = self.bowExtractor.getVocabulary()#numClusters x 128
        hists = []
        counter = 0
        for arr in descriptors:
            hist = np.zeros((self.numClusters, 1))#histogram for each image with respect to the clusters.
            for row in arr:
                #finding nearest cluster to construct visual word or codeblock
                i = np.argmin(np.sqrt(np.sum(np.power(vocuabulary - row.reshape(1, -1), 2), axis=1)), axis=0)
                hist[i, 0] += 1
            hists.extend(hist.reshape(1, -1))
            #labels_extends.extend([labels[counter]] * self.nfeatures)
            counter += 1
        return np.array(hists, np.float32), np.array(labels).reshape(-1, 1)


    def loadDescriptorsAndLabels(self, dataSize, windowSize=200, visualize=False):
        #Get 2 * dataSize of images
        labels = [0] * dataSize
        labels.extend([1] * dataSize)
        images = self.loadData(dataSize, windowSize, "cat", "cat.")   
        images.extend(self.loadData(dataSize, windowSize, "dog", "dog."))

        des, labels = self.getDescriptors(images, labels, visualize)
        return np.array(des, np.float32) , labels

    def loadDataRandomly(self, numImages, windowSize, showMe=False):
        images = []
        labels = []
        indeces = np.random.choice(12500, numImages, replace=False)
        typeAnimal = np.random.choice(2, numImages)
        lab = {0: "cat.", 1:"dog."}
        for ind in range(0, len(indeces)):

            try:
                img = cv2.imread(f"/kaggle/working/train/" + lab[typeAnimal[ind]] + str(indeces[ind]) + ".jpg", cv2.IMREAD_GRAYSCALE)

                #histogram equalization to remove skewness in the intensity
                img_center = np.zeros((windowSize, windowSize), dtype=np.ubyte)
                numStrides = [int(np.ceil(img.shape[0]/windowSize)), int(np.ceil(img.shape[1]/windowSize ))]
                numStrides[0] = 2 if numStrides[0] <= 1 else numStrides[0]
                numStrides[1] = 2 if numStrides[1] <= 1 else numStrides[1]
                #print(numStrides)
                img = img[::numStrides[0], ::numStrides[1]]
                img_center[:img.shape[0], :img.shape[1]] = img[:, :]
                img_center = cv2.equalizeHist(img_center)
                img_center = np.asarray(img_center, np.float32)
                means, stddev = cv2.meanStdDev(img_center)
                img_center -= means
                stddev = stddev if stddev != 0 else 1
                img_center /= stddev

                images.append(img_center)
                labels.append(typeAnimal[ind])

            except Exception as e:
                print(e)

        return images, labels

# Visualizing the Images with the Features Descriptions
Notice, that the images have a uniform shape in which I used the same shape for each image. But instead of taking the middle part of the image, I relied upon subsampling the image by striding within the image. Also, I utilized histogram equalization in order to minimize noise caused by non-uniform distribution of the intensity in the image.

In [ ]:
obj = LoadingImages(10, 64, 256)
cats = obj.loadData(10, 200, "cat", "cat.", "train", False)

gs = GridSpec(2, 5)
for row in range(0, 2):
    for col in range(0, 5):
        axes = plt.subplot(gs[row, col])
        axes.imshow(cats[row*2 + col], cmap="gray")
        axes.set_xticks([])
        axes.set_yticks([])
        axes.set_title("cat")
plt.show()

In [ ]:
obj = LoadingImages(10, 64, 256)
dogs = obj.loadData(10, 200, "dog", "dog.", "train", False)

gs = GridSpec(2, 5)
for row in range(0, 2):
    for col in range(0, 5):
        axes = plt.subplot(gs[row, col])
        axes.imshow(dogs[row*2 + col], cmap="gray")
        axes.set_xticks([])
        axes.set_yticks([])
        axes.set_title("cat")
plt.show()

In [ ]:
kps = []
des, labels = obj.getDescriptors(cats, [0]*len(cats), False, kps)
colored_imgs = []
for img, kp in zip(cats, kps):
    channels = []
    colored = None
    img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC3)
    colored = cv2.drawKeypoints(img, kp, img, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    colored_imgs.append(cv2.cvtColor(colored, cv2.COLOR_BGR2RGB))
    
gs = GridSpec(2, 5)
for row in range(0, 2):
    for col in range(0, 5):
        axes = plt.subplot(gs[row, col])
        axes.imshow(colored_imgs[row*2 + col], cmap="gray")
        axes.set_xticks([])
        axes.set_yticks([])
        axes.set_title("cat")
plt.show()

In [ ]:
kps = []
des, labels = obj.getDescriptors(cats, [0]*len(dogs), False, kps)
colored_imgs = []
for img, kp in zip(dogs, kps):
    channels = []
    colored = None
    img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC3)
    colored = cv2.drawKeypoints(img, kp, img, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    colored_imgs.append(cv2.cvtColor(colored, cv2.COLOR_BGR2RGB))
    
gs = GridSpec(2, 5)
for row in range(0, 2):
    for col in range(0, 5):
        axes = plt.subplot(gs[row, col])
        axes.imshow(colored_imgs[row*2 + col], cmap="gray")
        axes.set_xticks([])
        axes.set_yticks([])
        axes.set_title("dog")
plt.show()

# Utility methods For Training and Predicitions

In [ ]:
def train(obj, training_errors, validation_errors, epochs, batch_size, valid_size, batchs_num, validation_size, train_size, model_type, windowSize, decisionValue, noiseUpdate=0.05, nu=0.9, gamma=0.005, kernel="rbf", C=0.8):
    # cat = 0, dog = 1
    np.random.seed(42)
    if model_type == "svm":
        model = NuSVC(kernel=kernel, nu=nu, gamma=gamma)
        modelTemp = NuSVC(kernel=kernel, nu=nu, gamma=gamma)
    else:
        model = LogisticRegression(solver='sag', penalty="l2", C=C, warm_start=True, max_iter=400, n_jobs=2)#default parameters
        modelTemp = LogisticRegression(solver='sag', penalty="l2", C=C, warm_start=True, max_iter=400, n_jobs=2)#default parameters
    counter = 0
    prevError = np.finfo(np.float32).min

    for epoch in range(0, epochs):
        training_error_avg = []
        #Clear clusters at each epoch.
        obj.resetCounters()
        #obj.bowKmeansClusters.clear()
        #obj.bowExtractor.setVocabulary(np.array([]))
        for batch in range(0, batchs_num):
            #transform = PolynomialFeatures(degrees, interaction_only=True)
            des, labels = obj.loadDescriptorsAndLabels(int(batch_size/2), windowSize) #batch_sizex128
            des, labels = obj.getBOW(des, labels, False)
            #des = transform.fit_transform(des)
            indeces = np.arange(len(des))
            np.random.shuffle(indeces)
            if modelType == "svm":
                modelTemp.fit(des[indeces], labels[indeces]);
                y_pred = modelTemp.predict(des[indeces])
            else:
                modelTemp.fit(des[indeces], labels[indeces]); 
                #y_pred = model.predict(des)
                y_pred = list(map(lambda v: 0 if v[0] > decisionValue else 1, modelTemp.predict_proba(des[indeces])))
            try:
                cnf = confusion_matrix(y_pred, labels[indeces])
                training_error_avg.append(np.sum(np.diag(cnf))/np.sum(cnf))
                #training_error_avg.append(log_loss(labels[indeces], y_pred))
                print(f"{epoch}-{batch} the accuracy is {np.sum(np.diag(cnf))/np.sum(cnf)}")
                print(cnf)

            except Exception as e:
                print(e)
                pass

            if training_error_avg[-1] + np.random.normal(0) * noiseUpdate > prevError :
                print("Update model")
                model = modelTemp
                prevError = training_error_avg[-1]
            else:
                print("Previous model is better")
                counter += 1

            if counter > 15:
                print(f"No update for the parameter for {counter} passes")
                break

        if batchs_num * batch_size < train_size - 1 and counter <= 15:
        # transform = PolynomialFeatures(degrees, interaction_only=True)
            des, labels = obj.loadDescriptorsAndLabels(int(train_size - batchs_num * batch_size), windowSize)
            des, labels = obj.getBOW(des, labels, False)
            indeces = np.arange(len(des))
            np.random.shuffle(indeces)
            if modelType == "svm":    
                #des = transform.fit_transform(des)
                modelTemp.fit(des[indeces], labels[indeces]);
                y_pred = modelTemp.predict(des[indeces]) 
            else:
                modelTemp.fit(des[indeces], labels[indeces]); 
                y_pred = list(map(lambda v: 0 if v[0] > decisionValue else 1, modelTemp.predict_proba(des[indeces])))
                
            try:
                cnf = confusion_matrix(y_pred, labels[indeces])
                training_error_avg.append(np.sum(np.diag(cnf))/np.sum(cnf))
                print(f"Last accuracy is {np.sum(np.diag(cnf))/np.sum(cnf)}") 
                print(cnf)

            except Exception as e:
                    print(e) 
                    pass

            if training_error_avg[-1] > prevError:
                print("Update support Vectors")
                #model = modelTemp
                prevError = training_error_avg[-1]
        training_errors.append(np.mean(training_error_avg))

        #For validation set
        labels = []
        y_pred = []
        for valid_batch in range(0, int(validation_size/valid_size)):
            des, labels_r = obj.loadDescriptorsAndLabels(valid_size, windowSize) #numb_key_ptx128
            des, labels_r = obj.getBOW(des, labels_r, False)
            #transform = PolynomialFeatures(degrees, interaction_only=True)
            #des = transform.fit_transform(des[:])
            labels.extend(labels_r)
            if modelType == "logistic":
                y_pred_r = list(map(lambda v: 0 if v[0] > decisionValue else 1, model.predict_proba(des)))
            else:
                y_pred_r = model.predict(des)

            y_pred.extend(y_pred_r)
        if len(y_pred) < validation_size:
            des, labels_r = obj.loadDescriptorsAndLabels(valid_size, windowSize) #numb_key_ptx128
            des, labels_r = obj.getBOW(des, labels_r, False)
            #transform = PolynomialFeatures(degrees, interaction_only=True)
            #des = transform.fit_transform(des[:])
            labels.extend(labels_r)
            if modelType == "logistic":
                y_pred_r = list(map(lambda v: 0 if v[0] > decisionValue else 1, model.predict_proba(des)))
            else:
                y_pred_r = model.predict(des)
            y_pred.extend(y_pred_r)
        try:
            cnf = confusion_matrix(y_pred, labels)
            validation_errors.append(np.sum(np.diag(cnf))/np.sum(cnf))
            print(cnf)
            print(f"{epoch} the validation accuracy is {np.sum(np.diag(cnf))/np.sum(cnf)}") 

        except Exception as e:
            print(e)
            pass

        if epoch%2 == 0:
            print(f"#epoch: {epoch} and the accuracy training is {training_errors[epoch]}")    
            print(f"#epoch: {epoch} and the accuracy validation is {validation_errors[epoch]}")


    return model, training_errors, validation_errors

def prediction(obj, model, modelType, decisionValue, numImages, windowSize, datasetType, trainOrValid=True, visualize=False):
    batch_size = 1024
    numBatches = int(numImages/batch_size)
    if datasetType == "train" and not trainOrValid:
        obj.counts["cat"] = train_size
        obj.counts["dog"] = train_size
    else:
        obj.resetCounters()

    labels = []
    y_pred = []
    counter = 1 
    print(obj.counts)
    for batch in range(0, numBatches):
        if datasetType == "train":
            des, labels_r = obj.loadDescriptorsAndLabels(batch_size, windowSize) #numb_key_ptx128
            des, labels_r = obj.getBOW(des, labels_r, False)
        else:
            images = obj.loadData(batch_size, windowSize, "test", "", "test", False, 0) #batch_sizex128
            des, labels_r = obj.getDescriptors(images, [-1]*batch_size, False)
            des, labels_r = obj.getBOW(des, [-1]*batch_size, False)
        labels.extend(labels_r)
        if modelType == "logistic":
            y_pred_r = list(map(lambda v: 0 if v[0] > decisionValue else 1, model.predict_proba(des)))
        else:
            y_pred_r = model.predict(des)
        y_pred.extend(y_pred_r)
        if visualize:
            for img in images:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                cv2.putText(img, 'cat' if y_pred_r[counter] == 0 else 'dog', (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255))
                cv2.imshow("Test_image" + str(counter), img)
                counter += 1
                if cv2.waitKey(0) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    break
        print(f"Batch number {batch}, number of processed data {(batch + 1) * batch_size}")
    if len(y_pred) < numImages:
        if datasetType == "train":
            des, labels_r = obj.loadDescriptorsAndLabels(numImages - len(y_pred), windowSize) #numb_key_ptx128
            des, labels_r = obj.getBOW(des, labels_r, False)
        else:
            images = obj.loadData(numImages - len(y_pred), windowSize, "test", "", "test", False, 0) #batch_sizex128
            des, labels_r = obj.getDescriptors(images, [-1]*batch_size, False)
            des, labels_r = obj.getBOW(des, [-1]*batch_size, False)
        if modelType == "logistic":
            y_pred_r = list(map(lambda v: 0 if v[0] > decisionValue else 1, model.predict_proba(des)))
        else:
            y_pred_r = model.predict(des)
        y_pred.extend(y_pred_r)
        
    return labels, y_pred

# Setting up the unique Clusters

In [ ]:
train_validation_ratio = 0.85
trainingSize = 12500
train_size = int(trainingSize * train_validation_ratio)
validation_size = int(trainingSize * (1 - train_validation_ratio))
windowSize = 200
nfeatures = 256
numClusters = 64

obj = LoadingImages(trainingSize, numClusters, nfeatures)#30 clusters within an image

# Setting clusters early on
#numObservations = numClusters * 100
numObservations = 2048

images, labels = obj.loadDataRandomly(numObservations, windowSize)
des, labels = obj.getDescriptors(images, labels)
des, labels = obj.getBOW(np.array(des, np.float32), labels, True)

print(f"#clusters is {obj.bowExtractor.descriptorSize()}")

# Setting up The Model

In [ ]:
modelType = "logistic"
training_errors = []
validation_errors = []
batch_size = 2048#For svm
batch_size = 512
batchs_num = int(train_size/batch_size)
valid_size = 256
epochs = 1
decisionValue = 0.52

model, training_errors, validation_errors = train(obj, training_errors, validation_errors, epochs, batch_size, 
                                                  valid_size, batchs_num, validation_size, train_size, modelType,
                                                  windowSize, decisionValue)

# Model Performance on the training set

In [ ]:
labels, y_pred = prediction(obj, model, modelType, decisionValue, train_size, windowSize, "train", True)

cnf = confusion_matrix(y_pred, labels)
print(cnf)
print(f"Accuracy: {np.sum(np.diag(cnf))/np.sum(cnf)}")

# Model Performance on the validation set

In [ ]:
labels, y_pred = prediction(obj, model, modelType, decisionValue, trainingSize - train_size, windowSize, "train", False)

cnf = confusion_matrix(y_pred, labels)
print(cnf)
print(f"Accuracy: {np.sum(np.diag(cnf))/np.sum(cnf)}")

# Model Performance on the test set

In [ ]:
labels, y_pred = prediction(obj, model, modelType, decisionValue, 12500, windowSize, "test", True, False)

In [ ]:
results = pd.DataFrame(np.c_[list(range(1, len(y_pred) + 1)), y_pred], columns=["id", "label"])
results.to_csv("submission.csv", index=False)
results.head()

In [ ]:
# To download file, from https://www.kaggle.com/rtatman/download-a-csv-file-from-a-kernel
# from IPython.display import HTML
# import base64
# html = '<a  href="{filename}" target="_blank">{title}</a>'
# html = html.format(title="file",filename="submission.csv")
# HTML(html)

# Suggested Improvements
* We can use HOG instead of SIFT
* We can still rely on SIFT but instead of using BoVW, we can utilize the key descriptors themselves to construct the feature vector, but this will exponentially increase the number of parameters based on the number of features that you specify.
* Choosing better regularization values, increase the number of epoch, and it is always recommended to use ensemble of models for the prediction step
* As you can see from the accuracy for both validation and test set that deep learning is way better than traditional CV when you have a large dataset. But remember that this model had only 64 parameters(i.e the number of clusters), and this is significantly less than the millions of parameters that you see in CNN.